modelo 1

funciona correcto clip -> mejorar descripciones con indice

vit -> descripciones no encajan

ocr -> lo hace correcto

In [ ]:
import torch
from PIL import Image
import easyocr

# ---------- CONFIGURACIÓN ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
image_path = "porta_esquerra.jpeg"  # Cambia aquí tu imagen

# ---------- 1. CLIP ----------
print("\n🧠 CLIP: comparando con descripciones conocidas...")

import open_clip  # Asegúrate de haber hecho: pip install open_clip_torch

# Cargar modelo CLIP desde open_clip
model_clip, _, preprocess_clip = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
tokenizer = open_clip.get_tokenizer('ViT-B-32')
model_clip.to(device)

image_clip = preprocess_clip(Image.open(image_path)).unsqueeze(0).to(device)

descriptions = [
    "Pantalla central del sistema de infoentretenimiento",
    "Volante con botones multifunción",
    "Guantera del lado del copiloto",
    "Puerta derecha delantera",
    "Zona trasera con asiento y cinturón",
    "Botón de arranque del vehículo",
    "Panel de control de climatización",
]

text_tokens = tokenizer(descriptions).to(device)

with torch.no_grad():
    image_features = model_clip.encode_image(image_clip)
    text_features = model_clip.encode_text(text_tokens)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarities = (image_features @ text_features.T).squeeze(0)
    best_idx = similarities.argmax().item()

print(f"🔍 CLIP sugiere: '{descriptions[best_idx]}'")
for i, sim in enumerate(similarities):
    print(f"   - {descriptions[i]:<50} → score: {sim:.3f}")

# ---------- 2. ViT-GPT2 (en lugar de BLIP-2) ----------
print("\n🧾 ViT-GPT2: generando descripción automática...")

from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

model_blip = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning").to(device)
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

image = Image.open(image_path).convert("RGB")
pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values.to(device)

with torch.no_grad():
    output_ids = model_blip.generate(pixel_values, max_length=50, num_beams=4)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

print(f"💬 Descripción generada por ViT-GPT2: {caption}")

# ---------- 3. OCR (EasyOCR) ----------
print("\n🔠 OCR: texto detectado en la imagen...")
reader = easyocr.Reader(['en', 'es'], gpu=torch.cuda.is_available())
results = reader.readtext(image_path)

if not results:
    print("   (No se detectó texto visible)")
else:
    for (bbox, text, prob) in results:
        print(f"   - '{text}' (confianza: {prob:.2f})")

gitbase -> 1 bien 1 mal, muy especifico, escaso

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

# ---------- CONFIGURACIÓN ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
image_path = "porta_esquerra.jpeg"  # Cambia aquí tu imagen

# ---------- GIT base: Generar descripción ----------
print("\n🧾 GIT base: generando descripción automática...")

# Cargar el modelo y el procesador
processor = AutoProcessor.from_pretrained("microsoft/git-base")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base").to(device)

# Preprocesar la imagen
image = Image.open(image_path).convert("RGB")
inputs = processor(images=image, return_tensors="pt").to(device)

# Generar caption
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_length=50)
    caption = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

# Mostrar resultado
print(f"💬 Descripción generada por GIT base: {caption}")






blipbase -> no acaba de generar buenas descripciones, se nota que describe coche

In [ ]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# ---------- CONFIGURACIÓN ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
image_path = "porta_esquerra.jpeg"  # Cambia aquí tu imagen

# ---------- BLIP base: Generar descripción ----------
print("\n🧾 BLIP (base): generando descripción automática...")

# Cargar modelo y procesador
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# Cargar y procesar imagen
image = Image.open(image_path).convert('RGB')
inputs = processor(image, return_tensors="pt").to(device)

# Generar caption
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=50)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

# Mostrar resultado
print(f"💬 Descripción generada por BLIP base: {caption}")

ofa -> no llego ha ejecutarse